# 규제
심층 신경망은 전형적으로 수만, 때로는 수백만 개의 파라미터를 갖고 있음. 이 때문에 네트워크의 자유도가 매우 높음.  
덕분에 대규모의 복잡한 데이터셋을 학습할 수 있지만 **학습 데이터셋에 과대적합될 위험이 매우 크다는 것을 의미.**  
**규제**가 필요하다. 앞서 **조기 종료, 배치 정규화**같은 것을 이미 다뤄봤음. 이번엔  
- l1, l2 규제
- 드롭아웃
- 맥스-노름(max-norm) 규제

를 알아보겠음.

---
## l1, l2 규제


In [4]:
from tensorflow import keras

In [8]:
layer = keras.layers.Dense(100, activation="elu",
                          kernel_initializer="he_normal",
                          kernel_regularizer=keras.regularizers.l2(0.01))

layer = keras.layers.Dense(100, activation="elu",
                          kernel_initializer="he_normal",
                          kernel_regularizer=keras.regularizers.l1(0.01))

layer = keras.layers.Dense(100, activation="elu",
                          kernel_initializer="he_normal",
                          kernel_regularizer=keras.regularizers.l1_l2(0.01))

**l2, l1, l1_l2** 함수들은 학습하는 동안 규제 손실을 계산하기 위해 각 스텝에서 호출되는 규제 객체를 반환함.  
이 손실은 최종 손실에 합산됨.

In [12]:
from functools import partial

RegularizedDense = partial(keras.layers.Dense,
                          activation="elu",
                          kernel_initializer="he_normal",
                          kernel_regularizer=keras.regularizers.l2(0.01))

In [13]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    RegularizedDense(300),
    RegularizedDense(100),
    RegularizedDense(10, activation="softmax",
                    kernel_initializer="glorot_uniform")
])

일반적으로 네트워크의 모든 은닉층에 동일한 활성함수, 동일한 초기화 전략을 사용하거나 모든 층에 동일한 규제를 적용함.  
이는 코드를 고치기 어렵게 만듦.  
이를 피하기 위해 파이썬의 **functools.partial()** 함수를 사용하여 기본 매개변수 값을 사용하여 함수 호출을 감쌀 수 있음.

---
## 드롭아웃
**Dropout**은 심층 신경망에서 가장 인기있는 규제 기법 중 하나임.  
최고 성능을 내는 신경망조차도 드롭 아웃을 적용해서 정확도를 1~2% 높일 수 있음.  
>매 학습 스텝에서 각 뉴런(입력 뉴런은 포함, 출력 뉴런은 제외)은 임시적으로 드롭아웃될 확률 p를 가짐.  
**드롭아웃 비율** p는 보통 10~50%로 설정. 순환 신경망에선 20~30%, 합성곱 신경망에선 40~50%에 가까움.  
학습이 끝나면 드롭아웃은 작동하지 않음.

### 드롭아웃 효과
- 드롭아웃으로 학습된 뉴런은 이웃 뉴런에 적응될 수 없어서 자기 자신이 유용한 뉴런이 되야함.
- 몇 개의 입력 뉴런에만 지나치게 의존할 수 없어서 모든 입력 뉴런에 주의를 기울임. 즉 입력값의 작은 변화에 덜 민감해짐(일반화 성능 업)

드롭아웃의 능력을 이해하는 또 다른 방법은 각 학습 스텝마다 고유한 네트워크가 생성된다고 보면 됨.  
10000번 학습을 진행하면 10000개의 다른 신경망이 생성되는 것인데 이 신경망은 대부분의 가중치를 공유하고 있어 완전 독립되지 않음.  
하지만 그럼에도 모두 다름. 결과적으로 최종 신경망은 이 모든 신경망들을 평균한 **앙상블**로 볼 수 있음.

### 드롭아웃 주의점
만약 p=50%로 설정하면 학습이 아닌 테스트동안엔 하나의 뉴런이 학습때보다 (평균적으로) 2배 많은 입력 뉴런과 연결됨.  
이렇게 되면 각 뉴런이 학습한 것보다 거의 두 배 많은 입력 신호를 받아서 잘 동작하지 않을 것임.  
이런 점을 보상하기 위해 학습 후에 각 뉴런의 **연결 가중치에 0.5를 곱할 필요가 있음**  
>일반적으로 말하면 학습이 끝나면 각 뉴런의 연결 가중치에 **보존 확률 (1-p)를 곱해야 함**

In [14]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
])

- **모델이 과대적합되었다면** : 드롭아웃 비율 증가
- **모델이 과소적합되었담녀** : 드롭아웃 비율 감소

> 또한 많은 최신 신경망 구조에서는 **마지막 은닉층 뒤에만 드롭아웃을 사용한다고 함**

>드롭 아웃은 **수렴을 상당히 느리게 만드는 경향이 있지만** 적절하게 튜닝하면 **훨씬 좋은 모델을 만듦**  
따라서 일반적으로 추가적인 시간과 노력을 기울일 가치가 있음

> 드롭아웃은 학습하는 동안에만 활성화되므로 **학습 손실과 검증 손실을 비교하면 안됨**  
특히 비슷한 학습 손실과 검증 손실을 얻었더라도 모델이 학습 데이터셋에 과대적합되었을 수 있음.  
따라서 **드롭 아웃을 빼고(예를 들어 학습이 끝나고) 학습 손실을 평가해야 함**

---
## 몬테 카를로 드롭아웃
야린 갤, 주빈 가라마니의 2016년 논문에서 **몬테 카를로 드롭아웃**이라는 기법을 소개함.  
>몬테 카를로 드롭아웃은 학습된 드롭아웃 모델을 재학습하거나 전혀 수정하지 않고 성능을 크게 향상시킬 수 있다고 함

In [16]:
fashion_mnist = keras.datasets.fashion_mnist
(x_train_full, y_train_full), (x_test, y_test) = fashion_mnist.load_data()

x_val, x_train = x_train_full[: 5000]/255.0, x_train_full[5000: ]/255.0
y_val, y_train = y_train_full[: 5000], y_train_full[5000: ]
x_test = x_test/255.0

4423680/4422102 [==============================] - 0s 0us/step


In [17]:
import numpy as np

y_probs = np.stack([model(x_test, training=True) for sample in range(100)])
y_proba = y_probs.mean(axis=0)

In [21]:
y_probs.shape, y_proba.shape

((100, 10000, 10), (10000, 10))

In [20]:
x_test.shape

(10000, 28, 28)

우선 **training=True**로 설정하여 드롭아웃을 활성화시키고 테스트셋에서 100번의 예측을 만듦.  
드롭아웃이 활성화되었기 때문에 이 100개의 예측은 모두 다름.  
이 100개의 예측을 **axis=0**으로 평균하여 원래 테스트셋에 대해 **predict**한 것과 동일한 크기의 배열을 얻게 됨.
> 이게 끝임. 즉 드롭아웃으로 만든 예측을 평균하면 일반적으로 **드롭아웃 없이 예측한 하나의 결과보다 안정적임**

In [24]:
model.compile(loss="sparse_categorical_crossentropy",
             optimizer="sgd",
             metrics=["accuracy"])

model.fit(x_train, y_train, epochs=100,
             validation_data=(x_val, y_val),
             callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Epoch 1/100
1719/1719 [==============================] - 4s 2ms/step - loss: 1.0069 - accuracy: 0.6428 - val_loss: 0.5146 - val_accuracy: 0.8248
Epoch 2/100
1719/1719 [==============================] - 3s 2ms/step - loss: 0.6049 - accuracy: 0.7819 - val_loss: 0.4666 - val_accuracy: 0.8356
Epoch 3/100
1719/1719 [==============================] - 3s 2ms/step - loss: 0.5469 - accuracy: 0.8017 - val_loss: 0.4446 - val_accuracy: 0.8440
Epoch 4/100
1719/1719 [==============================] - 3s 2ms/step - loss: 0.5225 - accuracy: 0.8105 - val_loss: 0.4247 - val_accuracy: 0.8504
Epoch 5/100
1719/1719 [==============================] - 3s 2ms/step - loss: 0.5070 - accuracy: 0.8142 - val_loss: 0.4082 - val_accuracy: 0.8574
Epoch 6/100
1719/1719 [==============================] - 3s 2ms/step - loss: 0.4877 - accuracy: 0.8231 - val_loss: 0.4002 - val_accuracy: 0.8624
Epoch 7/100
1719/1719 [==============================] - 3s 2ms/step - loss: 0.4871 - accuracy: 0.8235 - val_loss: 0.3960 - val_ac

1719/1719 [==============================] - 3s 2ms/step - loss: 0.3575 - accuracy: 0.8666 - val_loss: 0.3102 - val_accuracy: 0.8858
Epoch 58/100
1719/1719 [==============================] - 3s 2ms/step - loss: 0.3554 - accuracy: 0.8674 - val_loss: 0.3053 - val_accuracy: 0.8898
Epoch 59/100
1719/1719 [==============================] - 3s 2ms/step - loss: 0.3596 - accuracy: 0.8689 - val_loss: 0.3070 - val_accuracy: 0.8866
Epoch 60/100
1719/1719 [==============================] - 3s 2ms/step - loss: 0.3584 - accuracy: 0.8675 - val_loss: 0.3089 - val_accuracy: 0.8856
Epoch 61/100
1719/1719 [==============================] - 3s 2ms/step - loss: 0.3554 - accuracy: 0.8675 - val_loss: 0.3053 - val_accuracy: 0.8874
Epoch 62/100
1719/1719 [==============================] - 3s 2ms/step - loss: 0.3477 - accuracy: 0.8701 - val_loss: 0.3030 - val_accuracy: 0.8876
Epoch 63/100
1719/1719 [==============================] - 3s 2ms/step - loss: 0.3536 - accuracy: 0.8685 - val_loss: 0.3043 - val_accuracy

In [28]:
np.round(model.predict(x_test[:1]), 2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.05, 0.  , 0.94]],
      dtype=float32)

In [29]:
y_probs = np.stack([model(x_test, training=True) for sample in range(100)])
y_proba = y_probs.mean(axis=0)

In [31]:
np.round(y_probs[:, :1], 2)

array([[[0.  , 0.  , 0.  , 0.  , 0.  , 0.05, 0.  , 0.02, 0.  , 0.93]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.04, 0.  , 0.03, 0.  , 0.93]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.32, 0.  , 0.02, 0.  , 0.66]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.11, 0.  , 0.89]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.02, 0.  , 0.07, 0.  , 0.91]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.12, 0.  , 0.88]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.02, 0.  , 0.55, 0.  , 0.43]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.98]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.02, 0.  , 0.07, 0.  , 0.91]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.02, 0.  , 0.98]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.02, 0.  , 0.98]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.21, 0.  , 0.78]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.01, 0.  , 0.98]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.16, 0.  , 0.02, 0.  , 0

In [32]:
np.round(y_proba[:1], 2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.05, 0.  , 0.09, 0.  , 0.86]],
      dtype=float32)

확실히 predict로 출력한 확률보다 드롭아웃들의 평균이 출력한 확률이 더 그럴듯 함.  
이 확률 추정의 표준 분포도 확인해볼 수 있음

In [34]:
y_std = y_probs.std(axis=0)
np.round(y_std[:1], 2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.09, 0.  , 0.1 , 0.  , 0.14]],
      dtype=float32)

In [40]:
y_pred = np.argmax(model.predict(x_test), axis=1)
acc_pred = np.sum(y_pred == y_test) / len(y_test)
acc_pred

0.8777

In [41]:
y_pred_dropout = np.argmax(y_proba, axis=1)
acc_pred_dropout = np.sum(y_pred_dropout == y_test) / len(y_test)
acc_pred_dropout

0.8784

오 그리고 모델의 정확도도 아주 조금 상향됨 ㅎㅎ

> 하지만 **학습하는 동안 다르게 작동하는(배치 정규화같은) 층을 갖고 있다면** 이와 같이 학습 모드를 강제로 설정해서는 안됨.  
대신 Dropout층을 다음과 같은 **MCDropout** 클래스를 정의하여 바꿔주면 됨

In [42]:
class MCDropout(keras.layers.Dropout):
    def call(self, inputs):
        return super().call(inputs, training=True)

즉 **training**매개변수를 강제로 True로 설정하여 학습이 끝나도 드롭아웃이 활성화되도록 하는 것임.
>**MC드롭아웃은 모델의 성능을 높여주고 더 정확한 불확실성 추정을 제공**하는 좋은 기술임.  
그리고 학습하는 동안은 일반적인 드롭아웃처럼 수행하여 규제처럼 작용하기도 함.

---
## 맥스-노름 규제
> 이 방식은 각각의 뉴런에 대해 입력의 **연결 가중지 w의 l2 노름이 일정 값(r)을 넘지 않도록 제한함**

맥스-노름 규제는 전체 손실 함수에 규제 손실 항을 추가하지 않음.  
대신 일반적으로 매 훈련 스텝이 끝나고 w의 l2노름을 계산하고 필요하면 w의 스케일을 조정함.  
- **r을 줄이면** 규제의 양이 증가하여 과대적합을 감소시키는 데 도움이 됨.  

>맥스-노름 규제는 (배치 정규화를 사용하지 않았을 때) 불안정한 그레디언트 문제를 완화하는 데 도움을 줄 수 있음  
(그럼 배치정규화 쓰면 안써도 된다는 뜻??)

In [43]:
layer = keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal",
                          kernel_constraint=keras.constraints.max_norm(1.))

매 이터레이션마다 모델의 fit()메서드가 층의 가중치와 함께 max_norm()이 반환한 객체를 호출하고 스케일이 조정된 가중치를 반환받음.  
이 값을 사용하여 층의 가중치를 바꿈.  
사용자 정의 규제 함수를 정의하여 kernel_constraint 매개변수에 지정하여 편향을 규제할 수도 있음.  
>기본적으로 0으로 설정된 **axis** 매개변수가 있음  
**합성곱 신경망에 사용하려면** axis=[0, 1, 2] 이런식으로 적절하게 지정해야 함.